In [3]:
import scipy as sp
import numpy as np
import pandas as pd
import os,sys
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer,TfidfTransformer
import matplotlib.pyplot as plt
import seaborn as sns
datasets=r'D:\雲科\資料探勘\資料探勘第三次作業\mini_newsgroups'
from nltk.corpus import stopwords
import nltk
from sklearn.cluster import KMeans
from sklearn import cluster,metrics
import time
KKSTticks=time.process_time()
from sklearn.decomposition import TruncatedSVD

In [4]:
orderedList=os.listdir(datasets)
texts=[]
labels_index={}
labels=[]

In [5]:
for p in orderedList:
    mypath=os.path.join(datasets,p)
    if os.path.isdir(mypath):
        label_id=len(labels_index)
        labels_index[p]=label_id
        for fname in sorted(os.listdir(mypath)):
            if fname.isdigit():
                fpath=os.path.join(mypath,fname)

In [6]:
if sys.version_info<(3,):
    f=open(fpath)
else:
    f=open(fpath,encoding='latin-1')
    t=f.read()
    u=t.find('\n\n')
    if 0<u:
            t=t[u:]
    texts.append(t)
    f.close()
    labels.append(label_id)

In [7]:
news_df=pd.DataFrame({'texts':texts})
news_df['clean_doc']=news_df['texts'].str.replace("[^a-zA-Z#]"," ")
news_df['clean_doc']=news_df['clean_doc'].apply(lambda x:' '.join([w for w in x.split() if len(w)>3]))
# news_df
news_df['clean_doc']=news_df['clean_doc'].apply(lambda x: x.lower())
# news_df
nltk.download('stopwords')
# news_df
stop_words=stopwords.words('english')
# news_df
tokenized_doc=news_df['clean_doc'].apply(lambda x:x.split())
# news_df
tokenized_doc=tokenized_doc.apply(lambda x:[item for item in x if item not in stop_words])
news_df

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,texts,clean_doc
0,\n\nIn article <1993Apr27.073723.18577@csis.di...,article csis csiro csis csiro peter lamb write...


In [8]:
detokenized_doc=[]
for p in range(len(news_df)):
    t=' '.join(tokenized_doc[p])
    detokenized_doc.append(t)
    
# news_df['clean_doc']=detokenized_doc

aaalist = detokenized_doc[0].split(' ')

vectorizer = TfidfVectorizer(stop_words='english',
                            max_features=2000,
                            max_df=0.5,
                            smooth_idf=True)
X=vectorizer.fit_transform(aaalist)

In [9]:
svd_model=TruncatedSVD(n_components=20,algorithm='randomized',
                      n_iter=100,random_state=122)
svd_model.fit(X)

TruncatedSVD(n_components=20, n_iter=100, random_state=122)

In [10]:
X_topics=svd_model.fit_transform(X)

In [11]:
KKSTticks=time.process_time()
kmeans=KMeans(n_clusters=20)
kmeans.fit(X_topics)
cluster_labels=kmeans.labels_

KKEDticks=time.process_time()

print('KMEAN所花費的時間為:',KKEDticks-KKSTticks)

KMEAN所花費的時間為: 0.109375


In [12]:
def purity(X, cluster_labels):
    cluster = np.array(X)
    label = np. array(cluster_labels)
    indedata1 = {}
    for p in np.unique(label):
        indedata1[p] = np.argwhere(label == p)
    indedata2 = {}
    for q in np.unique(cluster):
        indedata2[q] = np.argwhere(cluster == q)

    count_all = []
    for i in indedata1.values():
        count = []
        for j in indedata2.values():
            a = np.intersect1d(i, j).shape[0]
            count.append(a)
        count_all.append(count)

    return sum(np.max(count_all, axis=0))/len(cluster)

In [13]:
purity(X_topics,cluster_labels)

20.094117647058823